In [37]:
#importing all the packages 
import numpy as np
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})
plt.rcParams.update({'font.size': 22})

import pandas as pd
from sklearn import datasets, linear_model
from sklearn.datasets import load_iris
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [38]:
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

### Loading and preprocessing the dataset

In [39]:
# we will use the handwritten MNIST digit dataset

digits_mnist = tf.keras.datasets.mnist.load_data()

In [40]:
(X_train_full, y_train_full), (X_test, y_test) = digits_mnist

In [41]:
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [42]:
X_train.shape


(55000, 28, 28)

In [7]:
X_train.dtype

dtype('uint8')

In [43]:
# scaling the pixel intensities
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.


### Defining the model Architecture 

In [44]:
# defining the model (one way to do it) 
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [ ]:
# same as above

#model = tf.keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=[28, 28]),
    #tf.keras.layers.Dense(300, activation="relu"),
    #tf.keras.layers.Dense(100, activation="relu"),
    #tf.keras.layers.Dense(10, activation="softmax")
#])

### Displaying the model Architecture 

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [46]:
# you can get weights of any layer in the model
hidden2 = model.layers[2]
weights, biases = hidden2.get_weights()

### Compiling the model 

In [49]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

### Training and Evaluating the model  

In [50]:
history = model.fit(X_train, y_train, epochs=30, verbose = 0, validation_data=(X_valid, y_valid))

In [ ]:
pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlim=[0, 29], ylim=[0, 1], grid=True, xlabel="Epoch",
    style=["r--", "r--.", "b-", "b-*"])
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)

In [ ]:
y_proba.round(2)

In [ ]:
y_pred = y_proba.argmax(axis=-1)
y_pred

In [ ]:
#check if our predictions are correct or not
plt.imshow(X_test[1], cmap= 'binary')
plt.axis('off')
plt.show()

In [ ]:
pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlim=[0, 29], ylim=[0, 1], grid=True, xlabel="Epoch",
    style=["r--", "r--.", "b-", "b-*"])
plt.show()

### Saving and loading the model

In [ ]:
model.save("keras_digits_model", save_format="tf")


In [ ]:
# extra code – show the contents of the my_keras_model/ directory
from pathlib import Path
for path in sorted(Path("keras_digits_model").glob("**/*")):
    print(path)

In [ ]:
model_load  = tf.keras.models.load_model("keras_digits_model")

### Model Callbacks

In [29]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_mnist_model", save_best_only=True)
history = model.fit(X_train, y_train, epochs=30,verbose =0, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb])

In [30]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=30,verbose =0, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, early_stopping_cb])


INFO:tensorflow:Assets written to: my_mnist_model/assets


INFO:tensorflow:Assets written to: my_mnist_model/assets


INFO:tensorflow:Assets written to: my_mnist_model/assets


INFO:tensorflow:Assets written to: my_mnist_model/assets


INFO:tensorflow:Assets written to: my_mnist_model/assets


INFO:tensorflow:Assets written to: my_mnist_model/assets


INFO:tensorflow:Assets written to: my_mnist_model/assets


INFO:tensorflow:Assets written to: my_mnist_model/assets


In [36]:
model = tf.keras.models.load_model("my_mnist_model") # rollback to best model
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0682 - accuracy: 0.9791


[0.06824850291013718, 0.9790999889373779]